In [31]:
import requests
from dotenv import dotenv_values

# Load the environment variables from the .env file
env_vars = dotenv_values("../local.env")

# Access the environment variables
api_key = env_vars['ALPHA_VANTAGE_API_KEY']

In [42]:
def fetchIntradayDataCSV(symbol: str, interval: str, month: str, api_key: str, result_file: str, append=False):
    print(f"Fetching data for {symbol} for the month of {month}...")

    api_domain = "www.alphavantage.co"
    function = "TIME_SERIES_INTRADAY"
    output_size = "full"
    url = "https://{}/query?function={}&symbol={}&interval={}&month={}&outputsize={}&apikey={}".format(
        api_domain, function, symbol, interval, month, output_size, api_key)

    r = requests.get(url)
    data = r.json()

    price_data = data.get("Time Series (1min)")

    if not price_data:
        print(f"Error: {data.get('Error Message')}")
        return

    if not append:
        with open(result_file, 'w') as f:
            f.write('Date,Open,High,Low,Close,Volume\n')

    with open(result_file, 'a') as f:
        for date, price in price_data.items():
            f.write(date + ',' + price['1. open'] + ',' + price['2. high'] + ',' +
                    price['3. low'] + ',' + price['4. close'] + ',' + price['5. volume'] + '\n')
    print(f"Data for {symbol} for the month of {month} fetched successfully.")


def getMonths(start: str, end: str):
    start_year, start_month = map(int, start.split('-'))
    end_year, end_month = map(int, end.split('-'))

    months = []

    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            if year == start_year and month < start_month:
                continue
            if year == end_year and month > end_month:
                break
            months.append(f"{year}-{month:02d}")

    return months

In [45]:
# create csv file
symbol = "AAPL"
interval = "1min"
month = "2021-02"

months = getMonths("2021-01", "2021-12")

result_file = "{}_intraday.csv".format(symbol)
with open(result_file, 'w') as f:
    f.write('Date,Open,High,Low,Close,Volume\n')
for month in months:
    fetchIntradayDataCSV(symbol, interval, month, api_key,
                         result_file, append=True)

In [48]:
# get data from csv file
import pandas as pd
data = pd.read_csv(result_file)

# sort data by date and save
data = data.sort_values(by='Date')
data.to_csv(result_file, index=False)
print(data.head())

                      Date     Open     High      Low    Close  Volume
16969  2021-01-04 04:00:00  130.834  131.031  130.521  131.011   25026
16968  2021-01-04 04:01:00  131.011  131.031  130.983  131.011     798
16967  2021-01-04 04:02:00  131.119  131.139  130.992  131.020     580
16966  2021-01-04 04:03:00  131.030  131.051  131.002  131.030     218
16965  2021-01-04 04:04:00  131.286  131.306  131.189  131.236    1492
